<a href="https://colab.research.google.com/github/gopichandchalla16/infosys-internship-real-time-industry-insight-system/blob/main/Infosys_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infosys Springboard Internship Project  
## Real-Time Industry Insight & Strategic Intelligence System

**Team Members:** Gopichand, Anshika, Janmejay, Vaishnavi  

This notebook implements an end-to-end AI-powered competitive intelligence platform that:

- Monitors real-time industry trends via Google News RSS
- Synthesizes mock social sentiment (Twitter-like posts)
- Builds a unified text corpus for market & sentiment analysis
- Uses Gemini LLM–based sentiment scoring with a local fallback
- Trains time-series forecasting models (Prophet with ARIMA fallback)
- Generates BUY/SELL/HOLD opportunity & threat alerts
- Delivers an executive dashboard using Plotly subplots

---

## High-Level Architecture

**1. Market & Sentiment Analysis Engine**
- Source: Google News RSS + mock Twitter-style posts
- Unified corpus builder
- Sentiment scoring:
  - Primary: Gemini 2.0 Flash (via `google-generativeai`)
  - Fallback: Local lexicon-based sentiment analyzer

**2. Trend & Positioning Forecasting System**
- Source: `yfinance` (1-year OHLCV data)
- Primary model: Prophet (daily forecasting)
- Fallback model: ARIMA (if Prophet import or fit fails)
- Output: 7-day forecast with confidence intervals

**3. Opportunity & Threat Alert Module**
- Combines:
  - Forecasted 7-day price movement
  - Aggregated sentiment score (–100 to +100)
- Outputs: BUY / SELL / HOLD signals
- Notification:
  - Primary: Slack (if webhook configured)
  - Fallback: Console print

**4. Strategic Insight Dashboard (Single Cell)**
- Plotly subplots:
  - Historical vs forecast (with confidence band)
  - Sentiment gauge (–100 to +100)
  - 7-day forecast bars
  - Executive summary metrics table
  - Signal badge annotation

---

## Delivery Plan (Sprints)

- **Sprint 1:** Data sourcing & cleaning
- **Sprint 2:** LLM sentiment, news processing & unified corpus
- **Sprint 3:** Forecasting & signal computation
- **Sprint 4:** Slack alerts & executive dashboard

In [17]:
# Install core dependencies
!pip install -q yfinance plotly feedparser prophet google-generativeai statsmodels

In [31]:
# IMPORTS & GLOBAL SETTINGS

import os
import json
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import feedparser
import yfinance as yf

from statsmodels.tsa.arima.model import ARIMA
from plotly.subplots import make_subplots
import plotly.graph_objects as go


from google.colab import userdata

def load_secret(name: str) -> str:
    """
    Safely load secrets from the new Colab Secret Manager.
    Always returns a string ("" if not available).
    """
    try:
        val = userdata.get(name)
        return val.strip() if val else ""
    except:
        return ""

# Load secrets
ALPHA_VANTAGE_API_KEY = load_secret("ALPHA_VANTAGE_API_KEY")
GEMINI_API_KEY        = load_secret("GEMINI_API_KEY")
SLACK_WEBHOOK_URL     = load_secret("SLACK_WEBHOOK_URL")

# Display loaded key status
print("Alpha Vantage Key Loaded:", bool(ALPHA_VANTAGE_API_KEY))
print("Gemini Key Loaded:", bool(GEMINI_API_KEY))
print("Slack Webhook Loaded:", bool(SLACK_WEBHOOK_URL))

# ============================================================
# PROPHET INITIALIZATION
# ============================================================
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception:
    PROPHET_AVAILABLE = False

# ============================================================
# GEMINI LLM INITIALIZATION
# ============================================================
try:
    import google.generativeai as genai
    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
        GEMINI_MODEL = genai.GenerativeModel("gemini-2.0-flash")
        GEMINI_ENABLED = True
    else:
        GEMINI_ENABLED = False
except Exception:
    GEMINI_ENABLED = False


random.seed(42)
np.random.seed(42)

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 160)

print("Prophet available:", PROPHET_AVAILABLE)
print("Gemini enabled:", GEMINI_ENABLED)
print("Environment initialized.")

Alpha Vantage Key Loaded: True
Gemini Key Loaded: True
Slack Webhook Loaded: True
Prophet available: True
Gemini enabled: True
Environment initialized.


## Sprint 1: Data Sourcing & Cleaning

In this sprint we:

1. Capture user/company configuration (company name + ticker).
2. Fetch 1-year historical price data from `yfinance`.
3. Retrieve latest news articles from Google News RSS.
4. Generate mock Twitter-like posts to simulate social sentiment.
5. Build a unified, clean corpus ready for sentiment analysis.


In [33]:
# Company Input Cell with Reliable Ticker Auto-Detection (Fixed Version)

import yfinance as yf
import requests

default_company_name = "Tesla, Inc."
default_ticker = "TSLA"

# Banned fake names/patterns
INVALID_COMPANY_KEYWORDS = {
    "abc", "xyz", "test", "testing", "demo", "sample",
    "qwerty", "asdf", "fake", "dummy", "123", "456"
}

def is_invalid_company_name(name: str) -> bool:
    name_clean = name.strip().lower()
    for bad in INVALID_COMPANY_KEYWORDS:
        if bad in name_clean:
            return True
    if name_clean.isnumeric() or len(name_clean) < 2:
        return True
    return False


def search_ticker_by_company(company_name: str):
    """
    Uses Yahoo Finance search API with required headers.
    Returns (ticker, longName) or (None, None).
    """
    try:
        query = company_name.replace(" ", "+")
        url = f"https://query2.finance.yahoo.com/v1/finance/search?q={query}"

        headers = {
            "User-Agent": "Mozilla/5.0",
            "Accept": "application/json"
        }

        response = requests.get(url, headers=headers, timeout=5)
        data = response.json()

        if "quotes" not in data or len(data["quotes"]) == 0:
            return None, None

        # pick first EQUITY result
        for item in data["quotes"]:
            if item.get("quoteType") == "EQUITY":
                symbol = item.get("symbol")
                longname = item.get("longname") or item.get("shortname") or symbol
                return symbol, longname

        return None, None
    except Exception:
        return None, None


def verify_real_ticker(ticker: str) -> bool:
    """
    Verifies the ticker by checking for real market data.
    """
    try:
        t = yf.Ticker(ticker)
        hist = t.history(period="1mo")
        return not hist.empty
    except:
        return False


# -------------------------------
#          USER INPUT
# -------------------------------
try:
    user_company_name = input(f"Enter company name [{default_company_name}]: ").strip()
except EOFError:
    user_company_name = ""

COMPANY_INPUT = user_company_name if user_company_name else default_company_name

# Step 1 — Validate company name
if is_invalid_company_name(COMPANY_INPUT):
    print(f"Invalid company name detected: {COMPANY_INPUT}. Reverting to default.")
    COMPANY_NAME = default_company_name
    TICKER = default_ticker
else:
    # Step 2 — Auto-detect ticker from Yahoo Finance
    ticker, longname = search_ticker_by_company(COMPANY_INPUT)

    if ticker is None:
        print(f"No valid ticker found for '{COMPANY_INPUT}'. Reverting to default.")
        COMPANY_NAME = default_company_name
        TICKER = default_ticker
    else:
        # Step 3 — Verify ticker truly exists in yfinance
        if verify_real_ticker(ticker):
            COMPANY_NAME = longname
            TICKER = ticker.upper()
        else:
            print(f"Ticker '{ticker}' is not valid. Reverting to default.")
            COMPANY_NAME = default_company_name
            TICKER = default_ticker

print(f"Using company: {COMPANY_NAME} ({TICKER})")

Enter company name [Tesla, Inc.]: Tesla
Using company: Tesla, Inc. (TSLA)


In [34]:
# Uninstall pyppeteer to resolve the dependency conflict, as it's not a core requirement for the current project scope.
!pip uninstall -y pyppeteer

In [35]:
# Upgrade websockets to resolve dependency conflicts
!pip install --upgrade 'websockets>=15.0.1,<16.0.0' --quiet

## Company Profile & Summary Enrichment

This section enhances the competitive-intelligence system by retrieving official and public background information about the selected company:

1. **Wikipedia Summary**
2. **Yahoo Finance Business Summary**
3. **General Company Metadata**
4. **Sector & Industry Classification**
5. **Confidence-level fallback system**

This ensures the dashboard and insight engine have high-quality contextual information.


In [36]:
# Imports for summaries
import wikipedia
import yfinance as yf

def get_wikipedia_summary(company_name: str, sentences: int = 4) -> str:
    """
    Fetch a short Wikipedia summary for the company.
    Handles disambiguation & page errors gracefully.
    """
    try:
        wikipedia.set_lang("en")
        results = wikipedia.search(company_name)

        if not results:
            return "No Wikipedia page found for this company."

        # First result is usually correct
        page_title = results[0]

        try:
            summary = wikipedia.summary(page_title, sentences=sentences)
            return summary
        except wikipedia.exceptions.DisambiguationError as e:
            # Use first valid option
            try:
                fallback = e.options[0]
                return wikipedia.summary(fallback, sentences=sentences)
            except:
                return "Wikipedia page is ambiguous or unavailable."
        except:
            return "Wikipedia summary not available."
    except Exception:
        return "Wikipedia summary not available."


def get_yfinance_company_summary(ticker: str) -> dict:
    """
    Get company business summary, sector, industry, and key metadata from Yahoo Finance.
    """
    try:
        t = yf.Ticker(ticker)
        info = t.info

        return {
            "longName": info.get("longName"),
            "sector": info.get("sector"),
            "industry": info.get("industry"),
            "country": info.get("country"),
            "website": info.get("website"),
            "businessSummary": info.get("longBusinessSummary"),
        }
    except Exception:
        return {
            "longName": None,
            "sector": None,
            "industry": None,
            "country": None,
            "website": None,
            "businessSummary": None,
        }


# Fetch company metadata
wiki_summary = get_wikipedia_summary(COMPANY_NAME)
yf_summary = get_yfinance_company_summary(TICKER)

print("Company Name:", COMPANY_NAME)
print("Ticker:", TICKER)
print("\n=== Wikipedia Summary ===\n")
print(wiki_summary)

print("\n=== Yahoo Finance Company Summary ===\n")
print(yf_summary.get("businessSummary", "Not available"))

print("\nSector:", yf_summary.get("sector"))
print("Industry:", yf_summary.get("industry"))
print("Country:", yf_summary.get("country"))
print("Website:", yf_summary.get("website"))

Company Name: Tesla, Inc.
Ticker: TSLA

=== Wikipedia Summary ===

Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures, and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors.

=== Yahoo Finance Company Summary ===

Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise

## Company Profile Summary

This cell organizes all retrieved information into a clean, human-readable summary block.


In [37]:
# Clean formatted display block

from IPython.display import Markdown, display

company_profile_md = f"""
# **Company Strategic Profile**

### **{COMPANY_NAME} ({TICKER})**

---

## **Wikipedia Summary**
{wiki_summary}

---

## **Yahoo Finance Business Overview**
{yf_summary.get("businessSummary", "Not available")}

---

### **Key Metadata**
- **Sector:** {yf_summary.get("sector")}
- **Industry:** {yf_summary.get("industry")}
- **Country:** {yf_summary.get("country")}
- **Website:** {yf_summary.get("website")}

"""

display(Markdown(company_profile_md))


# **Company Strategic Profile**

### **Tesla, Inc. (TSLA)**

---

## **Wikipedia Summary**
Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures, and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors.

---

## **Yahoo Finance Business Overview**
Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; purchase financing and leasing services; services for electric vehicles through its company-owned service locations and Tesla mobile service technicians; and vehicle limited warranties and extended service plans. The Energy Generation and Storage segment engages in the design, manufacture, installation, sale, and leasing of solar energy generation and energy storage products, and related services to residential, commercial, and industrial customers and utilities through its website, stores, and galleries, as well as through a network of channel partners. This segment also provides services and repairs to its energy product customers, including under warranty; and various financing options to its residential customers. The company was formerly known as Tesla Motors, Inc. and changed its name to Tesla, Inc. in February 2017. Tesla, Inc. was incorporated in 2003 and is headquartered in Austin, Texas.

---

### **Key Metadata**
- **Sector:** Consumer Cyclical
- **Industry:** Auto Manufacturers
- **Country:** United States
- **Website:** https://www.tesla.com



Sprint 1: Market Data Acquisition

This section retrieves market data required for trend analysis and forecasting.

Key Components:
- 1-year OHLCV data from Yahoo Finance
- Basic market metrics (market cap, current price)
- Strict validation to prevent empty or faulty data

In [38]:
# Market Data Fetch Functions (Fully Fixed & Clean Output)

import yfinance as yf
import pandas as pd

def fetch_historical_data(ticker: str, period: str = "1y", interval: str = "1d") -> pd.DataFrame:
    """
    Fetch historical OHLCV data using yfinance.
    Explicitly sets auto_adjust=False to avoid future warnings.
    Ensures clean, validated DataFrame output.
    """
    df = yf.download(
        ticker,
        period=period,
        interval=interval,
        progress=False,
        auto_adjust=False    # FIX: Avoids FutureWarning
    )

    if df is None or df.empty:
        raise ValueError(f"No market data returned for ticker {ticker}")

    df = df.dropna().reset_index()
    df = df.rename(columns={"Date": "date"})
    return df


def fetch_market_metrics(ticker: str) -> dict:
    """
    Fetch key market metrics:
    - current market price
    - market cap (accurate method)
    """
    t = yf.Ticker(ticker)

    # Current Price
    hist = t.history(period="5d")
    last_price = float(hist["Close"].iloc[-1]) if not hist.empty else None

    # Market Cap (most reliable source)
    market_cap = None
    try:
        info = t.info
        market_cap = info.get("marketCap")
    except Exception:
        market_cap = None

    return {
        "current_price": last_price,
        "market_cap": market_cap
    }


# Fetch final data
market_df = fetch_historical_data(TICKER)
market_metrics = fetch_market_metrics(TICKER)

# Clean formatted output
print("==========================================")
print("          MARKET DATA SUMMARY")
print("==========================================")
print(f"Company: {COMPANY_NAME}")
print(f"Ticker: {TICKER}")
print(f"Historical rows: {len(market_df)}")
print(f"Date range: {market_df['date'].min().date()} → {market_df['date'].max().date()}")
print("------------------------------------------")
print(f"Current price: ${market_metrics['current_price']:.2f}" if market_metrics["current_price"] else "Current price: N/A")

# Format market cap
mc = market_metrics["market_cap"]
if mc:
    if mc >= 1e12:
        mc_str = f"${mc/1e12:.2f} Trillion"
    elif mc >= 1e9:
        mc_str = f"${mc/1e9:.2f} Billion"
    elif mc >= 1e6:
        mc_str = f"${mc/1e6:.2f} Million"
    else:
        mc_str = f"${mc:,}"
    print(f"Market cap: {mc_str}")
else:
    print("Market cap: N/A")

print("==========================================")

          MARKET DATA SUMMARY
Company: Tesla, Inc.
Ticker: TSLA
Historical rows: 251
Date range: 2024-12-10 → 2025-12-10
------------------------------------------
Current price: $447.33
Market cap: $1.49 Trillion


Sprint 1: News Feed (Google News RSS)

This module collects real-time news related to the selected company.

Features:
- Uses Google News RSS (safe & official)
- No rate-limit issues
- Clean text extraction
- Supports date parsing

In [39]:
import feedparser
from datetime import datetime
import pandas as pd

def build_google_news_url(query: str) -> str:
    q = query.replace(" ", "+")
    return f"https://news.google.com/rss/search?q={q}&hl=en-US&gl=US&ceid=US:en"


def fetch_google_news(query: str, max_items: int = 15) -> pd.DataFrame:
    """
    Fetch Google News RSS articles including:
    - title
    - summary
    - text
    - published_at
    - hyperlink to the source
    """
    url = build_google_news_url(query)
    feed = feedparser.parse(url)

    rows = []

    for entry in feed.entries[:max_items]:
        title = entry.get("title", "").strip()
        summary = entry.get("summary", "").strip()
        link = entry.get("link", "").strip()

        # published date
        try:
            pub_date = datetime(*entry.published_parsed[:6])
        except:
            pub_date = None

        # clean combined text
        text = f"{title}. {summary}".replace("\n", " ").strip()

        rows.append({
            "source": "news",
            "title": title,
            "summary": summary,
            "text": text,
            "link": link,
            "published_at": pub_date
        })

    return pd.DataFrame(rows)


# Run news fetcher
NEWS_QUERY = f"{COMPANY_NAME} stock"
news_df = fetch_google_news(NEWS_QUERY, max_items=10)

print("News articles fetched:", len(news_df))
news_df.head(5)

def make_clickable(link):
    if pd.isna(link) or link == "":
        return ""
    return f'<a href="{link}" target="_blank">Open Source</a>'


news_df_display = news_df.copy()
news_df_display["source_link"] = news_df_display["link"].apply(make_clickable)

from IPython.display import HTML
HTML(news_df_display.to_html(escape=False, index=False))

News articles fetched: 10


source,title,summary,text,link,published_at,source_link
news,Tesla: Mastering Fear And Greed In The Robotics Supercycle (Rating Downgrade) (TSLA) - Seeking Alpha,Tesla: Mastering Fear And Greed In The Robotics Supercycle (Rating Downgrade) (TSLA) Seeking Alpha,Tesla: Mastering Fear And Greed In The Robotics Supercycle (Rating Downgrade) (TSLA) - Seeking Alpha. Tesla: Mastering Fear And Greed In The Robotics Supercycle (Rating Downgrade) (TSLA) Seeking Alpha,https://news.google.com/rss/articles/CBMitgFBVV95cUxPTzVialJCSnBzakZkT1RfVGlPOV9yNEYxbzJYU1JpeDg5OU11OUV2bnJhZDhMck9yNlh0XzNfLVN5eU0zaUFtTlNOT0l3T1c5U3prdkpRRHRVUWxPZ1Q1RERYbTBhZkJ5M1ltQjlsMzN2THFfYjdqQlZZWElCZXRsV2dYbDhBdUxqSTJfRUs5cjQ3c1Ywbi11UG1HaWQzTVpoMUF4YlhJV0YwWGRYSWxiYzhNb01Ldw?oc=5,2025-12-09 14:49:24,Open Source
news,Tesla Stock Lifts 2025 Gains to 20% as Traders Eye Santa Rally - TradingView,Tesla Stock Lifts 2025 Gains to 20% as Traders Eye Santa Rally TradingView,Tesla Stock Lifts 2025 Gains to 20% as Traders Eye Santa Rally - TradingView. Tesla Stock Lifts 2025 Gains to 20% as Traders Eye Santa Rally TradingView,https://news.google.com/rss/articles/CBMirwFBVV95cUxQWFY5a2JWcnVkUjVGZU1IXzM1QVNLdVdLUUQxZEVaWGMtZEVvMTJfazE0akwyNnRqVDRWZlpXQTFXblE0anBNWnUzUDFGbDljS2RZZG5xTVRuQXBUWEJPSWpuRktqNG5lTzJ0N1Y3TG55QkdMSVp4UmNCcGlxaW9reTRNazR1cHphT3VWcVhkNVA0cjBWRmNlMHJvZGQ0a1ByMmNWdzNjYnhwZ1hMbENN?oc=5,2025-12-09 07:55:46,Open Source
news,"Cathie Wood’s ARK Sells Tesla Stock, Adds to Baidu and GeneDx Holdings - Investing.com","Cathie Wood’s ARK Sells Tesla Stock, Adds to Baidu and GeneDx Holdings Investing.com","Cathie Wood’s ARK Sells Tesla Stock, Adds to Baidu and GeneDx Holdings - Investing.com. Cathie Wood’s ARK Sells Tesla Stock, Adds to Baidu and GeneDx Holdings Investing.com",https://news.google.com/rss/articles/CBMiwwFBVV95cUxORF9FX0k1eEFJWkhKeGs3ZG9HTXBSY1Bqc2NTcTVSV2ExRVJIbjRKYzJuRHVKS2w1SjJXY0JJSjB5bmN3U0l1VEJhUkVibzVnclVUWF9Ibmp5VGplUWZDTXh5YlRUZmxWbjlnV2xGbVBFMGRrYkZlU21tT01LLUx2V0JGX2tMNnZhYjl5VHVSQkljLXZaSjZuUW1hVlRVR1pHd29PX0FhLVIwY2NrRnAyX0xLbEdWdWlYYkVxaTk0WFRqZEk?oc=5,2025-12-09 01:04:14,Open Source
news,A Low-Cost Model 3 Just Hit the Streets in Europe. Can That Help Turn Tesla Stock Around? - Barchart.com,A Low-Cost Model 3 Just Hit the Streets in Europe. Can That Help Turn Tesla Stock Around? Barchart.com,A Low-Cost Model 3 Just Hit the Streets in Europe. Can That Help Turn Tesla Stock Around? - Barchart.com. A Low-Cost Model 3 Just Hit the Streets in Europe. Can That Help Turn Tesla Stock Around? Barchart.com,https://news.google.com/rss/articles/CBMizAFBVV95cUxQVVkxR3pNSFo0V2VSZnhGcGN0U2dGMzh6T3JlSndBZmRwcm1KVlo5RXlTclJTRmpPbWhKbmJGNENYa1N5eV9ETXM2MWhYMUdWN3dycFFtdDBjekRPb3FGa0tkR0Rlel9sb3hSQm9vY19HRERjVFY4Nm02bVluelljVkFON1gyTEgxY2htMk14MWlhcGNSZ1BjbXo4S1BrV1ZWSkdHcFhvdEJoS1BwblNXR05xbU9zVXh6djJ4b0ZsQWJuNW40SkZDbzQ1MTg?oc=5,2025-12-08 21:59:15,Open Source
news,"‘The Dream Keeps Expanding,’ Says Top Investor About Tesla Stock - TipRanks","‘The Dream Keeps Expanding,’ Says Top Investor About Tesla Stock TipRanks","‘The Dream Keeps Expanding,’ Says Top Investor About Tesla Stock - TipRanks. ‘The Dream Keeps Expanding,’ Says Top Investor About Tesla Stock TipRanks",https://news.google.com/rss/articles/CBMilgFBVV95cUxOM0dBSFJ6UXZGajh0eWVrOVpPRnBfRmtmVUdxTlNURjJIa0xhUEVqSHdGNVZ4d0M2WXdnREU1NDl6UzZVRVppNS01UlhRWDRxcEwzb1FBcEhTTDVTZEc0M0pIODNWTmlfMEJTWFRiQXd4YVVsRFJBWG5UTm9uMVM0dzNUaTlIUm5EWkVHLU96RnpzRDZMMkE?oc=5,2025-12-10 08:43:26,Open Source
news,Tesla Stock Hits The Brakes: What's Driving Monday's Sell-Off? - Tesla (NASDAQ:TSLA) - Benzinga,Tesla Stock Hits The Brakes: What's Driving Monday's Sell-Off? - Tesla (NASDAQ:TSLA) Benzinga,Tesla Stock Hits The Brakes: What's Driving Monday's Sell-Off? - Tesla (NASDAQ:TSLA) - Benzinga. Tesla Stock Hits The Brakes: What's Driving Monday's Sell-Off? - Tesla (NASDAQ:TSLA) Benzinga,https://news.google.com/rss/articles/CBMiuwFBVV95cUxPLTcya3htRzU5MzFGYUk0Z2oxWWJvRG96VTA3dTVvQldvOEZGS0tLcXZIWXR0WEc0QVZfTTUtYUhNbHN2L

Sprint 1: Mock Twitter Data

Because real Twitter API access is restricted, this module generates realistic synthetic posts for sentiment analysis.

Includes:

- Positive / negative / neutral sentiment hints  
- Timestamp generation  
- Variety of realistic phrases

In [40]:
# Mock Twitter-like Posts

import random
from datetime import timedelta

random.seed(42)

POS_PHRASES = [
    "strong earnings", "bullish momentum", "positive guidance",
    "analysts optimistic", "record demand", "growth accelerating"
]

NEG_PHRASES = [
    "weak outlook", "bearish signals", "regulatory issues",
    "market concerns", "slowing demand", "negative pressure"
]

NEUTRAL_PHRASES = [
    "sideways movement", "no major change", "watching closely",
    "stable performance", "in line with expectations"
]

def generate_mock_tweets(company, days=5, posts_per_day=5):
    rows = []
    now = datetime.utcnow()

    for d in range(days):
        day = now - timedelta(days=d)
        for _ in range(posts_per_day):
            roll = random.random()
            if roll < 0.33:
                phrase = random.choice(POS_PHRASES); label="positive"
            elif roll < 0.66:
                phrase = random.choice(NEG_PHRASES); label="negative"
            else:
                phrase = random.choice(NEUTRAL_PHRASES); label="neutral"

            text = f"{company} shows {phrase} today. ({label})"

            rows.append({
                "source": "twitter",
                "title": "",
                "text": text,
                "published_at": day - timedelta(minutes=random.randint(0, 600))
            })
    return pd.DataFrame(rows)


tweets_df = generate_mock_tweets(COMPANY_NAME, days=5, posts_per_day=6)

print("Mock tweets:", len(tweets_df))
tweets_df.head(5)

Mock tweets: 30


/tmp/ipython-input-2841312628.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


,source,title,text,published_at
0,twitter,,"Tesla, Inc. shows weak outlook today. (negative)",2025-12-10 15:00:55.952502
1,twitter,,"Tesla, Inc. shows bullish momentum today. (pos...",2025-12-10 17:57:55.952502
2,twitter,,"Tesla, Inc. shows in line with expectations to...",2025-12-10 18:12:55.952502
3,twitter,,"Tesla, Inc. shows weak outlook today. (negative)",2025-12-10 19:11:55.952502
4,twitter,,"Tesla, Inc. shows bullish momentum today. (pos...",2025-12-10 11:04:55.952502


Sprint 1: Unified Corpus Builder

This module merges:
- Google News text  
- Mock Twitter text  

Into a **single unified text corpus** for sentiment analysis.

In [41]:
# Unified Corpus Builder

def build_unified_corpus(news_df: pd.DataFrame, tweets_df: pd.DataFrame) -> pd.DataFrame:
    df = pd.concat([news_df, tweets_df], ignore_index=True)
    df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")
    df = df.dropna(subset=["text"])
    df = df.sort_values("published_at", ascending=False).reset_index(drop=True)
    return df


corpus_df = build_unified_corpus(news_df, tweets_df)

print("Unified corpus size:", len(corpus_df))
corpus_df.head(5)

Unified corpus size: 40


,source,title,summary,text,link,published_at
0,twitter,,NaN,"Tesla, Inc. shows weak outlook today. (negative)",NaN,2025-12-10 19:11:55.952502
1,twitter,,NaN,"Tesla, Inc. shows in line with expectations to...",NaN,2025-12-10 18:12:55.952502
2,twitter,,NaN,"Tesla, Inc. shows bullish momentum today. (pos...",NaN,2025-12-10 17:57:55.952502
3,twitter,,NaN,"Tesla, Inc. shows slowing demand today. (negat...",NaN,2025-12-10 16:18:55.952502
4,news,"Tesla (TSLA) Live Share price, Invest From Ind...","<a href=""https://news.google.com/rss/articles/...","Tesla (TSLA) Live Share price, Invest From Ind...",https://news.google.com/rss/articles/CBMibkFVX...,2025-12-10 16:12:54.000000


## Sprint 2: Sentiment Engine + Alpha Vantage Market Data

In this sprint we:

1. Enhance market data sourcing by integrating **Alpha Vantage** as an additional stock data provider (with automatic fallback to `yfinance`).
2. Build a robust **sentiment engine**:
   - Primary: Gemini LLM (if `GEMINI_API_KEY` is set).
   - Fallback: Local lexicon-based sentiment analyzer.
3. Apply sentiment scoring to the unified corpus (news + mock Twitter).
4. Compute an aggregate sentiment score in the range **–100 to +100**.

In [42]:
# Alpha Vantage Market Data Integration (Fixed for Google Colab Secrets)

import os
import requests
from datetime import datetime, timedelta
import pandas as pd

# The ALPHA_VANTAGE_API_KEY is already loaded from Colab Secrets in cell EfYiBw9Wp5MX
# No need to re-read it here. The global variable is already set.

def fetch_alpha_vantage_daily(ticker: str, api_key: str, lookback_days: int = 365) -> pd.DataFrame:
    """
    Fetch daily adjusted close prices from Alpha Vantage.
    Returns DataFrame with columns: ['date', 'Close'].
    """
    if not api_key:
        raise ValueError("Alpha Vantage API key not found.") # Changed from 'environment' to be more general

    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": ticker,
        "outputsize": "full",
        "apikey": api_key
    }

    resp = requests.get(url, params=params, timeout=10)
    data = resp.json()

    if "Time Series (Daily)" not in data:
        raise ValueError(f"Alpha Vantage error for {ticker}: {data}")

    ts = data["Time Series (Daily)"]
    records = []
    cutoff = datetime.utcnow().date() - timedelta(days=lookback_days + 5)

    for date_str, vals in ts.items():
        dt = datetime.strptime(date_str, "%Y-%m-%d").date()
        if dt < cutoff:
            continue
        close_price = float(vals["5. adjusted close"])
        records.append({"date": datetime.combine(dt, datetime.min.time()), "Close": close_price})

    df = pd.DataFrame(records).sort_values("date").reset_index(drop=True)
    return df


# SAFE SWITCH BETWEEN ALPHA VANTAGE AND YFINANCE
try:
    # Use the globally available ALPHA_VANTAGE_API_KEY that was loaded in EfYiBw9Wp5MX
    if ALPHA_VANTAGE_API_KEY:
        alpha_df = fetch_alpha_vantage_daily(TICKER, ALPHA_VANTAGE_API_KEY)
        market_df = alpha_df
        print(f"Alpha Vantage data successfully loaded for {TICKER}.")
    else:
        print("Alpha Vantage key missing. Using yfinance instead.")
except Exception as e:
    print(f"Alpha Vantage fetch failed: {e}")
    print("Using yfinance instead.")

print("Final market_df rows:", len(market_df))
print("Final market_df date range:", market_df['date'].min().date(), "→", market_df['date'].max().date())

Alpha Vantage fetch failed: Alpha Vantage error for TSLA: {'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}
Using yfinance instead.
Final market_df rows: 251
Final market_df date range: 2024-12-10 → 2025-12-10


## Sprint 2 — Sentiment Analysis Engine

This module performs hybrid sentiment analysis using:

### 1. **Gemini 2.0 Flash**
- High-accuracy sentiment scoring (range: –100 to +100)
- Used when `GEMINI_API_KEY` is available

### 2. **Local Fallback Sentiment Engine**
- Uses lexicon-based keyword scoring
- Ensures the notebook still works without LLM access

### Output:
- Sentiment score per document
- Overall aggregated sentiment score

In [49]:
# ======================================================
# LOCAL FALLBACK SENTIMENT ANALYZER (RE-ADDING)
# ======================================================

POS_WORDS = [
    "growth", "strong", "bullish", "positive", "optimistic",
    "uptrend", "profit", "beats", "high demand", "surge"
]

NEG_WORDS = [
    "weak", "bearish", "negative", "downtrend", "loss",
    "regulatory", "lawsuit", "slowing", "fraud", "concern"
]

def local_sentiment(text: str) -> int:
    """
    Returns sentiment score between -100 and +100.
    Simple lexicon-based scoring.
    """
    text_l = text.lower()
    score = 0

    for w in POS_WORDS:
        if w in text_l:
            score += 10

    for w in NEG_WORDS:
        if w in text_l:
            score -= 10

    return max(-100, min(100, score))

In [52]:
# ======================================================
# SAFE SENTIMENT ENGINE (NO 429 ISSUES - RE-ADDING)
# ======================================================

def compute_sentiment_for_corpus(df):
    scores = []

    for _, row in df.iterrows():
        text = row["text"]

        # Use Gemini ONLY for news (low volume)
        if row["source"] == "news" and GEMINI_ENABLED:
            try:
                scores.append(gemini_sentiment(text))
            except:
                scores.append(local_sentiment(text))
        else:
            # Tweets and fallback use local sentiment
            scores.append(local_sentiment(text))

    df["sentiment"] = scores
    return df

corpus_df = compute_sentiment_for_corpus(corpus_df)
print("Sentiment (rate-safe) computed. Sample:")
print(corpus_df.head())

Sentiment (rate-safe) computed. Sample:
    source                                              title                                            summary  \
0  twitter                                                                                                   NaN   
1  twitter                                                                                                   NaN   
2  twitter                                                                                                   NaN   
3  twitter                                                                                                   NaN   
4     news  Tesla (TSLA) Live Share price, Invest From Ind...  <a href="https://news.google.com/rss/articles/...   

                                                text                                               link               published_at  sentiment  
0   Tesla, Inc. shows weak outlook today. (negative)                                                NaN 2025-12-10 19:1

In [51]:
# ======================================================
# GEMINI SENTIMENT ANALYZER (STRUCTURED OUTPUT)
# ======================================================

import time

def gemini_sentiment(text: str) -> int:
    """
    Uses Gemini 2.0 Flash to classify sentiment in range -100 to +100.
    Falls back to local sentiment if API fails.
    """
    if not GEMINI_ENABLED:
        return local_sentiment(text)

    try:
        prompt = f"""
        You are a sentiment analysis model.
        Analyze the sentiment of the following market-related text.
        Output ONLY an integer between -100 (very negative) and +100 (very positive).

        Text:
        {text}
        """

        # Add a short delay to mitigate rate limiting
        time.sleep(1) # Increased delay to 1 second
        result = GEMINI_MODEL.generate_content(prompt)
        val = int(result.text.strip())
        return max(-100, min(100, val))

    except Exception:
        return local_sentiment(text)

##Sprint 3 — Forecasting, Sentiment Aggregation & Signal Generation

This section prepares the historical market data and executes the full forecasting and signal-generation pipeline required for real-time strategic intelligence.

### **1. Time-Series Data Preparation for Forecasting**

The historical market data (`market_df`) is transformed into a Prophet-compatible structure with:

* **`ds`** → timestamp
* **`y`** → closing price

This formatting enables Prophet to recognize the temporal patterns for forecasting.

### **2. Forecasting Engine (Prophet → ARIMA Fallback)**

The system attempts to train a **Prophet model** using the prepared time-series dataset to generate a **7-day forward price forecast**, including:

* Predicted price (`yhat`)
* Lower confidence interval (`yhat_lower`)
* Upper confidence interval (`yhat_upper`)

If Prophet is unavailable or fails due to data constraints, the engine automatically falls back to an **ARIMA model**, ensuring uninterrupted forecasting capability.

### **3. Sentiment Fusion from Multi-Source Corpus**

The aggregated sentiment score is calculated from the unified text corpus (`corpus_df`), which includes:

* Google News articles
* Mock Twitter-style posts

Sentiment is normalized to a range of **–100 to +100**, representing:

* Strong bearish pressure
* Neutral sentiment
* Strong bullish momentum

### **4. Integrated BUY / SELL / HOLD Signal Engine**

The system combines:

* **Forecasted price movement (direction & magnitude)**
* **Aggregate sentiment score**

to derive a final **opportunity or threat signal**, classified as:

* **BUY** → Positive sentiment + upward forecast
* **SELL** → Negative sentiment + downward forecast
* **HOLD** → Mixed, weak, or neutral conditions

### **5. Executive Output Summary**

The following artifacts are produced for the dashboard:

* 7-day forward forecast (with confidence bands)
* Forecast trend direction
* Normalized sentiment score
* Final trading signal (BUY/SELL/HOLD)

This results in a unified, AI-driven strategic insight enabling more informed decision-making across competitive, financial, and market intelligence workflows.

In [53]:
prophet_df = market_df[['date', 'Close']].copy()
prophet_df = prophet_df.rename(columns={'date': 'ds', 'Close': 'y'})
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print("Prophet-ready DataFrame head:")
print(prophet_df.head())
print("\nProphet-ready DataFrame info:")
prophet_df.info()

Prophet-ready DataFrame head:
Price          ds           y
Ticker                   TSLA
0      2024-12-10  400.989990
1      2024-12-11  424.769989
2      2024-12-12  418.100006
3      2024-12-13  436.230011
4      2024-12-16  463.019989

Prophet-ready DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   (ds, )     251 non-null    datetime64[ns]
 1   (y, TSLA)  251 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.1 KB


In [54]:
prophet_df.columns = ['ds', 'y']

# ==================================================================
# Prophet Forecasting
# ==================================================================
forecast_days = 7
prophet_forecast_df = pd.DataFrame()
arima_forecast_df = pd.DataFrame()

if PROPHET_AVAILABLE:
    try:
        print("Attempting to train Prophet model...")
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        future = model.make_future_dataframe(periods=forecast_days)
        prophet_forecast_df = model.predict(future)

        print("Prophet model trained and forecast generated successfully.")

    except Exception as e:
        print(f"Prophet failed: {e}. Falling back to ARIMA.")
        PROPHET_AVAILABLE = False # Disable Prophet for this run if it failed
else:
    print("Prophet not available, falling back to ARIMA.")

# ==================================================================
# ARIMA Fallback Forecasting
# ==================================================================
if not PROPHET_AVAILABLE:
    try:
        print("Attempting to train ARIMA model...")
        # Use only the 'y' column for ARIMA, ensure it's numeric
        arima_data = prophet_df['y'].values
        # ARIMA parameters (p,d,q) - common starting point. Tune for better performance.
        # d=1 for differencing, p and q for AR and MA components
        # Using a simple (5,1,0) for demonstration
        arima_model = ARIMA(arima_data, order=(5,1,0))
        arima_model_fit = arima_model.fit()

        # Generate forecast for the next `forecast_days`
        arima_predictions = arima_model_fit.predict(start=len(arima_data), end=len(arima_data) + forecast_days - 1)

        # Create a DataFrame for ARIMA forecast
        last_date = prophet_df['ds'].max()
        future_dates = [last_date + timedelta(days=i) for i in range(1, forecast_days + 1)]

        # Create a series for yhat_lower and yhat_upper for ARIMA (simplified for now)
        # A more robust solution would involve calculating actual confidence intervals for ARIMA
        # For simplicity, we'll just set them as a fixed percentage around the prediction
        lower_bound = arima_predictions * 0.95 # 5% lower
        upper_bound = arima_predictions * 1.05 # 5% upper

        arima_forecast_df = pd.DataFrame({
            'ds': future_dates,
            'yhat': arima_predictions,
            'yhat_lower': lower_bound,
            'yhat_upper': upper_bound
        })
        print("ARIMA model trained and forecast generated successfully.")
    except Exception as e:
        print(f"ARIMA failed: {e}.")

# Use the relevant forecast DataFrame
final_forecast_df = prophet_forecast_df if PROPHET_AVAILABLE else arima_forecast_df

# Display forecast if successful
if not final_forecast_df.empty:
    print(f"\n7-Day Price Forecast for {COMPANY_NAME} ({TICKER}):")
    print(final_forecast_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(forecast_days))

# ==================================================================
# Aggregate Sentiment Score
# ==================================================================
aggregate_sentiment_score = corpus_df['sentiment'].mean()
# Normalize to -100 to +100 range, assuming individual sentiments are already in that range
# If individual scores are e.g., -10 to +10, this step is important for correct scaling
# If scores are already -100 to +100, mean directly gives desired range
# Given local_sentiment returns -100 to +100, the mean is already normalized.

print(f"\nAggregate Sentiment Score: {aggregate_sentiment_score:.2f} (Range: -100 to +100)")


Attempting to train Prophet model...


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Prophet model trained and forecast generated successfully.

7-Day Price Forecast for Tesla, Inc. (TSLA):
            ds        yhat  yhat_lower  yhat_upper
251 2025-12-11  459.620952  423.266776  497.868619
252 2025-12-12  463.079607  427.584720  500.851489
253 2025-12-13  455.922876  419.043997  493.718610
254 2025-12-14  456.816323  420.600184  494.755687
255 2025-12-15  465.825468  429.458121  501.827814
256 2025-12-16  465.781811  428.844610  504.671884
257 2025-12-17  469.264947  431.386100  505.694405

Aggregate Sentiment Score: 0.75 (Range: -100 to +100)


In [55]:
import numpy as np

# ==================================================================
# Opportunity & Threat Alert Module
# ==================================================================

# 1. Calculate 7-day price change from the forecast
if not final_forecast_df.empty:
    # Get the last historical closing price from prophet_df
    last_historical_price = prophet_df['y'].iloc[-1]
    # Get the forecasted closing price for the last day of the forecast period
    forecasted_price_7_day = final_forecast_df['yhat'].iloc[-1]

    price_change_7_day = ((forecasted_price_7_day - last_historical_price) / last_historical_price) * 100
    print(f"\nForecasted 7-day price change: {price_change_7_day:.2f}%")
else:
    price_change_7_day = 0.0
    print("\nNo forecast data available to calculate price change.")

# 2. Define thresholds and generate signal
SIGNAL_BUY_THRESHOLD_PRICE_CHANGE = 2.0  # % increase
SIGNAL_SELL_THRESHOLD_PRICE_CHANGE = -2.0 # % decrease
SIGNAL_BUY_THRESHOLD_SENTIMENT = 20.0 # score out of 100
SIGNAL_SELL_THRESHOLD_SENTIMENT = -20.0 # score out of 100

trading_signal = "HOLD"

if price_change_7_day > SIGNAL_BUY_THRESHOLD_PRICE_CHANGE and aggregate_sentiment_score > SIGNAL_BUY_THRESHOLD_SENTIMENT:
    trading_signal = "BUY"
elif price_change_7_day < SIGNAL_SELL_THRESHOLD_PRICE_CHANGE and aggregate_sentiment_score < SIGNAL_SELL_THRESHOLD_SENTIMENT:
    trading_signal = "SELL"

print(f"Generated Trading Signal: {trading_signal}")

# ==================================================================
# Executive Summary
# ==================================================================

# Extract relevant forecast data for summary
forecast_summary = final_forecast_df.tail(forecast_days)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_start_date = forecast_summary['ds'].min().strftime('%Y-%m-%d')
forecast_end_date = forecast_summary['ds'].max().strftime('%Y-%m-%d')

executive_summary_md = f"""
# **Strategic Intelligence Executive Summary**

## **{COMPANY_NAME} ({TICKER})**

---

### **Market Insights**
- **Current Price:** ${market_metrics['current_price']:.2f}
- **Market Cap:** {mc_str}

### **Forecasting**
- **Forecast Period:** {forecast_start_date} to {forecast_end_date}
- **Forecasted 7-Day Price Change:** {price_change_7_day:.2f}%
- **Predicted Price (End of Forecast):** ${forecasted_price_7_day:.2f} (with CI: ${forecast_summary['yhat_lower'].iloc[-1]:.2f} - ${forecast_summary['yhat_upper'].iloc[-1]:.2f})

### **Sentiment Analysis**
- **Aggregate Sentiment Score:** {aggregate_sentiment_score:.2f} (Range: -100 to +100)

### **Trading Signal**
## **{trading_signal}**

---
"""

from IPython.display import Markdown, display
display(Markdown(executive_summary_md))




Forecasted 7-day price change: 4.90%
Generated Trading Signal: HOLD



# **Strategic Intelligence Executive Summary**

## **Tesla, Inc. (TSLA)**

---

### **Market Insights**
- **Current Price:** $447.33
- **Market Cap:** $1.49 Trillion

### **Forecasting**
- **Forecast Period:** 2025-12-11 to 2025-12-17
- **Forecasted 7-Day Price Change:** 4.90%
- **Predicted Price (End of Forecast):** $469.26 (with CI: $431.39 - $505.69)

### **Sentiment Analysis**
- **Aggregate Sentiment Score:** 0.75 (Range: -100 to +100)

### **Trading Signal**
## **HOLD**

---


## Sprint 4 — Slack Alerts & Executive Dashboard

In this sprint, we finalize the system by adding real-time notifications and a unified executive visualization layer.

### 1. Slack Alert System
The model’s final trading signal and executive summary are sent directly to Slack using the configured `SLACK_WEBHOOK_URL`.  
If the webhook is not set or fails, the alert is safely printed to the console instead.  
This ensures critical BUY/SELL/HOLD updates reach decision-makers immediately.

### 2. Executive Dashboard (Plotly)
A single interactive dashboard is generated to present all key insights, including:

- **Historical vs. 7-Day Forecast Prices**  
  Line chart with future predictions and confidence intervals.

- **Sentiment Gauge**  
  Displays the aggregate sentiment score (–100 to +100).

- **Forecast Bar Chart**  
  Shows daily projected price levels for the next 7 days.

- **Executive Metrics Table**  
  Company info, current price, forecasted change, sentiment, and final trading signal.

- **Trading Signal Badge**  
  Color-coded BUY/SELL/HOLD indicator for quick interpretation.

This sprint delivers the complete intelligence layer—automatic alerts plus an interactive executive-ready dashboard.

In [56]:
import requests

def send_slack_notification(message: str):
    """
    Sends a message to Slack if SLACK_WEBHOOK_URL is configured,
    otherwise prints the message to the console.
    """
    if SLACK_WEBHOOK_URL:
        payload = {"text": message}
        try:
            response = requests.post(SLACK_WEBHOOK_URL, json=payload, timeout=5)
            response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
            print("Slack notification sent successfully.")
        except requests.exceptions.RequestException as e:
            print(f"Failed to send Slack notification: {e}")
            print("Fallback: Printing message to console.")
            print(message)
    else:
        print("Slack webhook URL not configured. Printing message to console.")
        print(message)

# Prepare the message combining trading signal and executive summary
notification_message = f"*Trading Signal Alert: {COMPANY_NAME} ({TICKER}) - {trading_signal}*\n\n" \
                       f"{executive_summary_md}"

# Send the notification
send_slack_notification(notification_message)


Slack notification sent successfully.


In [69]:
# STRATEGIC INTELLIGENCE DASHBOARD

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

# -----------------------------
# Safe access to key variables
# -----------------------------
# Forecast dataframe (fallback to forecast_df if final_forecast_df not defined)
_fc_df = globals().get("final_forecast_df", globals().get("forecast_df", None))
if _fc_df is None:
    raise ValueError("No forecast dataframe found (final_forecast_df or forecast_df).")

# Normalize forecast dataframe to expected schema
final_fc_df = _fc_df.copy()
if "ds" in final_fc_df.columns:
    final_fc_df = final_fc_df.rename(columns={"ds": "date"})

required_fc_cols = {"date", "yhat", "yhat_lower", "yhat_upper"}
if not required_fc_cols.issubset(final_fc_df.columns):
    raise ValueError(f"Forecast dataframe must contain columns {required_fc_cols}.")

# Historical data
if "date" not in market_df.columns or "Close" not in market_df.columns:
    raise ValueError("market_df must contain 'date' and 'Close' columns.")

hist_dates = market_df["date"]
hist_close = market_df["Close"]

# Split forecast into future only for 7-day block (if mixed)
last_hist_date = hist_dates.max()
future_fc = final_fc_df[final_fc_df["date"] > last_hist_date].copy()
if future_fc.empty:
    # If all forecast is in future anyway, just use full frame
    future_fc = final_fc_df.copy()

fc_dates = future_fc["date"]
fc_mean = future_fc["yhat"]
fc_low = future_fc["yhat_lower"]
fc_high = future_fc["yhat_upper"]

# Sentiment aggregation
if "sentiment" not in corpus_df.columns:
    raise ValueError("corpus_df must contain a 'sentiment' column.")

sent_value = corpus_df["sentiment"].mean() # Removed .item()
sent_norm = max(-100.0, min(100.0, sent_value))

# Forecast % change vs last close
last_price = hist_close.iloc[-1].item() # .item() is correct here as hist_close is a DataFrame with one column
forecast_pct = ((fc_mean.mean() - last_price) / last_price) * 100.0 # Removed .item() from fc_mean.mean()
final_fc_price = fc_mean.mean() # Removed .item()

# -----------------------------
# Executive KPI logic
# -----------------------------
def sentiment_label(score: float) -> str:
    if score >= 20:
        return "Bullish"
    elif score <= -20:
        return "Bearish"
    else:
        return "Neutral"

def trend_label(pct: float) -> str:
    if pct > 2:
        return "↑ Bullish"
    elif pct < -2:
        return "↓ Bearish"
    else:
        return "→ Sideways"

sent_text = sentiment_label(sent_norm)
trend_text = trend_label(forecast_pct)

# Final trading signal (support both naming styles)
signal_value = globals().get("trading_signal", globals().get("FINAL_SIGNAL", "HOLD"))

# Signal badge color
if str(signal_value).upper() == "BUY":
    signal_color = "green"
elif str(signal_value).upper() == "SELL":
    signal_color = "red"
else:
    signal_color = "orange"

# -----------------------------
# Market cap formatting
# -----------------------------
mc_raw = market_metrics.get("market_cap") if isinstance(market_metrics, dict) else None
if mc_raw:
    try:
        mc_raw = float(mc_raw)
        if mc_raw >= 1e12:
            market_cap_str = f"${mc_raw/1e12:.2f} Trillion"
        elif mc_raw >= 1e9:
            market_cap_str = f"${mc_raw/1e9:.2f} Billion"
        elif mc_raw >= 1e6:
            market_cap_str = f"${mc_raw/1e6:.2f} Million"
        else:
            market_cap_str = f"${mc_raw:,.0f}"
    except Exception:
        market_cap_str = "N/A"
else:
    market_cap_str = "N/A"

# -----------------------------
# Risk metrics (simple synthetic indices)
# -----------------------------
# Price volatility (last 60 days)
window_df = market_df.tail(60)
if len(window_df) > 1:
    returns = window_df["Close"].pct_change().dropna()
    price_vol = (returns.std() * np.sqrt(252) * 100.0).item()  # annualized % - ADDED .item()
else:
    price_vol = 0.0

# Forecast uncertainty (width of CI relative to price)
ci_width = (fc_high - fc_low).mean() if len(fc_high) else 0.0 # Removed .item()
forecast_uncertainty = (ci_width / final_fc_price * 100.0) if final_fc_price != 0 else 0.0

# Sentiment volatility
sent_vol = corpus_df["sentiment"].std() if len(corpus_df) > 1 else 0.0 # Removed .item()

# News intensity: count of news items
news_count = int((corpus_df["source"] == "news").sum()) if "source" in corpus_df.columns else 0

risk_labels = ["Price Volatility", "Forecast Uncertainty", "Sentiment Volatility", "News Flow Intensity"]
risk_values = [
    max(0.0, min(100.0, price_vol / 2.0)),                # scaled
    max(0.0, min(100.0, forecast_uncertainty)),           # %
    max(0.0, min(100.0, abs(sent_vol))),                  # already approx scaled
    max(0.0, min(100.0, news_count * 5.0)),               # 0,5,... scaled
]

# -----------------------------
# Sentiment breakdown table
# -----------------------------
# classify each row sentiment
def bucket_sentiment(v):
    if v >= 20:
        return "Bullish"
    elif v <= -20:
        return "Bearish"
    else:
        return "Neutral"

sent_buckets = corpus_df["sentiment"].apply(bucket_sentiment)
sent_counts = sent_buckets.value_counts().to_dict()

bull_cnt = sent_counts.get("Bullish", 0)
bear_cnt = sent_counts.get("Bearish", 0)
neut_cnt = sent_counts.get("Neutral", 0)

# -----------------------------
# Company logo (optional)
# -----------------------------
company_logo = globals().get("company_logo", None)

# -----------------------------
# Build dashboard layout
# -----------------------------
fig = make_subplots(
    rows=3,
    cols=2,
    specs=[
        [{"colspan": 2}, None],                              # Row 1: Price chart full width
        [{"type": "indicator"}, {"type": "table"}],          # Row 2: Sentiment gauge + Executive metrics
        [{"type": "bar"}, {"type": "table"}],                # Row 3: Risk bars + Sentiment breakdown
    ],
    subplot_titles=(
        "Historical Price vs 7-Day Forecast",
        "Aggregate Sentiment Gauge",
        "Executive Metrics Summary",
        "Risk Profile (Synthetic Indices)",
        "Sentiment Breakdown"
    )
)

# -------------------------------------------------------
# 1. Historical + Forecast Price Chart (Row 1, Col 1)
# -------------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=hist_dates,
        y=hist_close.iloc[:,0], # Ensure scalar access if hist_close is a DataFrame
        mode="lines",
        name="Historical Close"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=future_fc["date"],
        y=future_fc["yhat"],
        mode="lines+markers",
        name="Forecast (7D)"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=list(future_fc["date"]) + list(future_fc["date"][::-1]),
        y=list(future_fc["yhat_upper"]) + list(future_fc["yhat_lower"][::-1]),
        fill="toself",
        fillcolor="rgba(0, 100, 255, 0.18)",
        line=dict(color="rgba(0,0,0,0)"),
        name="Forecast CI",
        hoverinfo="skip"
    ),
    row=1, col=1
)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Price", row=1, col=1)

# -------------------------------------------------------
# 2. Sentiment Gauge (Row 2, Col 1)
# -------------------------------------------------------
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=sent_norm,
        title={"text": "Sentiment (−100 to 100)"},
        gauge={
            "axis": {"range": [-100, 100]},
            "bar": {"color": "darkblue"},
            "steps": [
                {"range": [-100, -25], "color": "rgba(255, 0, 0, 0.3)"},
                {"range": [-25, 25], "color": "rgba(200, 200, 200, 0.5)"},
                {"range": [25, 100], "color": "rgba(0, 200, 0, 0.3)"}
            ],
            "threshold": {
                "line": {"color": "black", "width": 3},
                "thickness": 0.7,
                "value": sent_norm
            }
        }
    ),
    row=2, col=1
)

# -------------------------------------------------------
# 3. Executive Metrics Table (Row 2, Col 2)
# -------------------------------------------------------
metrics_header = ["Metric", "Value"]

metrics_rows = [
    ["Company", COMPANY_NAME],
    ["Ticker", TICKER],
    ["Market Cap", market_cap_str],
    ["Current Price", f"${last_price:.2f}"],
    ["Forecasted Avg Price (7D)", f"${final_fc_price:.2f}"],
    ["7-Day Forecast Change", f"{forecast_pct:.2f}% ({trend_text})"],
    ["Aggregate Sentiment", f"{sent_norm:.2f} ({sent_text})"],
    ["Final Trading Signal", str(signal_value)],
]

fig.add_trace(
    go.Table(
        header=dict(values=metrics_header, fill_color="lightgrey", align="left"),
        cells=dict(values=list(zip(*metrics_rows)), align="left")
    ),
    row=2, col=2
)

# -------------------------------------------------------
# 4. Risk Profile Bar Chart (Row 3, Col 1)
# -------------------------------------------------------
fig.add_trace(
    go.Bar(
        x=risk_labels,
        y=risk_values,
        name="Risk Indices",
    ),
    row=3, col=1
)
fig.update_yaxes(title_text="Relative Risk (0–100)", row=3, col=1)

# -------------------------------------------------------
# 5. Sentiment Breakdown Table (Row 3, Col 2)
# -------------------------------------------------------
sent_table_header = ["Sentiment Bucket", "Count"]
sent_table_rows = [
    ["Bullish", bull_cnt],
    ["Neutral", neut_cnt],
    ["Bearish", bear_cnt],
]

fig.add_trace(
    go.Table(
        header=dict(values=sent_table_header, fill_color="lightgrey", align="left"),
        cells=dict(values=list(zip(*sent_table_rows)), align="left")
    ),
    row=3, col=2
)

# -------------------------------------------------------
# Global Layout, Title, and Signal Badge Annotation
# -------------------------------------------------------
title_text = f"Strategic Intelligence Dashboard — {COMPANY_NAME} ({TICKER})"

fig.update_layout(
    height=1100,
    showlegend=False,
    title=title_text,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Signal badge in the top-left overlay
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.01,
    y=1.12,
    text=f"Signal: {signal_value}",
    showarrow=False,
    font=dict(size=14, color="white"),
    align="left",
    bordercolor=signal_color,
    borderwidth=2,
    borderpad=4,
    bgcolor=signal_color,
    opacity=0.9,
)

if company_logo:
    fig.add_layout_image(
        dict(
            source=company_logo,
            xref="paper", yref="paper",
            x=0.99, y=1.14,
            sizex=0.12, sizey=0.12,
            xanchor="right", yanchor="top",
            layer="above"
        )
    )

fig.show()